## What are decorators?
Decorators are a way to modify or enhance functions (or classes) without changing their source code. They're essentially functions that wrap other functions.

### The basics
At their core, decorators use the fact that functions are first-class objects in Python - you can pass them around, return them, and assign them to variables.

Here's the simplest example:

In [32]:
def my_decorator(func):
    def wrapper():
        print("Before the function call.")
        result = func()
        if result is not None:
            print(result)
        print("After the function call.")
    return wrapper

def say_hello():
    return "Hello!"

say_hello = my_decorator(say_hello)
say_hello()

Before the function call.
Hello!
After the function call.


In [33]:
def my_decorator(func):
    def wrapper():
        print("Something before the function")
        func()
        print("Something after the function")
    return wrapper

def say_hello():
    print("Hello!")

say_hi = my_decorator(say_hello)
say_hi()

Something before the function
Hello!
Something after the function


仔细品一下，上面两个的差别！

再看看下面的过程：

In [ ]:
my_decorator(say_hello)()  # Directly calling the decorated function


Something before the function
Hello!
Something after the function


In [65]:
my_decorator(say_hello) # Getting the decorated function

<function __main__.my_decorator.<locals>.wrapper()>

In [68]:
my_decorator(say_hello())  # Calling the result of the decorator

Hello!


<function __main__.my_decorator.<locals>.wrapper()>

when you execute `my_decorator(say_hello())`, you called `say_hello()` first, which:
1. Executed the function immediately -> print "Hello!"
2. Return None since the function say_hello doesn't have a return statement
3. Pass the None to my_decorator
4. So you essentially did `my_decorator(None)`
5. the decorator still returned a wrapper function
6. if you try to call it by either:

```
result = my_decorator(say_hello())
result()
```

or just

```
my_decorator(say_hello())()
```

Then it would crash with:
```
TypeError: 'NoneType' object is not callable

**The key difference**

- `say_hello` is the function object (a reference)
- `say_hello()` calls the function and gives its return value

Think of it like this:
- `say_hello` is like giving someone a receipe
- `say_hello()` is like actually cooking the dish and handing them the food

**Decorators need the receipt (the function object), not the cooked dish (the return value)

In [ ]:
say_hello  # This is the function object itself - the actual "thing" that contains the code, 
           # You are just looking at it, not running it
           # Think of it like the receipt card itself (instructions on paper)

<function __main__.say_hello()>

In [ ]:
say_hello() # This executes/calls/runs the function
            # You are telling Python to actually run the code inside it
            # Actually cooking the recipe

<function __main__.say_hello()>

In [34]:
# Python provides a syntactic sugar to apply decorators using the "@" symbol.

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something before the function
Hello!
Something after the function


In [35]:
# To handle functions with arguments, we can use *args and **kwargs in the wrapper function.
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with arguments {args} and keyword arguments {kwargs}")
        result = func(*args, **kwargs)
        if result is not None:
            print(result)
        print(f"Finished calling {func.__name__}")
    return wrapper

@my_decorator
def add(x, y):
    return x + y

add(3, 5)

Calling add with arguments (3, 5) and keyword arguments {}
8
Finished calling add


In [43]:
# To handle functions with arguments, we can use *args and **kwargs in the wrapper function.
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with arguments {args} and keyword arguments {kwargs}")
        func(*args, **kwargs)
        print(f"Finished calling {func.__name__}")
    return wrapper

@my_decorator
def add(x, y):
    print(x + y)

add(3, 5)

Calling add with arguments (3, 5) and keyword arguments {}
8
Finished calling add


### Practical Examples

In [44]:

# Timing decorator:

import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

@timer
def slow_function():
    time.sleep(1)
    return "Done"

slow_function()

slow_function took 1.0050 seconds


'Done'

In [45]:
# Logging decorator:

def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

@log_calls
def multiply(x, y):
    return x * y

multiply(3, 4)

Calling multiply with args=(3, 4), kwargs={}
multiply returned 12


12

In [55]:
# Authentication decorator:

def require_auth(func):
    def wrapper(user, *args, **kwargs):
        if not user.get("authenticated"):
            raise PermissionError("User not authenticated")
        return func(user, *args, **kwargs)
    return wrapper

@require_auth
def view_profile(user):
    return f"Profile for {user['name']}"

print(view_profile({"name": "Alice", "authenticated": True}))  # Works

try:
    view_profile({"name": "Bob", "authenticated": False})  # Raises PermissionError
except PermissionError as e:
    print(e)

Profile for Alice
User not authenticated


### Decorators with arguments
You can create decorators that accept their own arguments by adding another layer:

In [56]:
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

Hello, Alice!
Hello, Alice!
Hello, Alice!


### Preserving metadata with functools.wraps
When you wrap a function, you lose its metadata (name, docstring, etc.). Use functools.wraps to preserve it:

In [57]:

from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example_function():
    """This is an example function."""
    pass
print(example_function.__name__)  # Output: example_function
print(example_function.__doc__)   # Output: This is an example function.

example_function
This is an example function.


In [60]:
# As a compare:

def my_decorator(func):
    def wrapper():
        print("Before the function call.")
        func()
        print("After the function call.")
    return wrapper

@my_decorator
def example_function():
    """This is an example function."""
    print("Hello!")

example_function()

print(example_function.__name__)  # Output: wrapper
print(example_function.__doc__)   # Output: None

Before the function call.
Hello!
After the function call.
wrapper
None


### Class-based decorators
You can also use classes as decorators:

In [61]:

class CountCalls:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call {self.count} of {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hi():
    print("Hi!")

say_hi()
say_hi()

Call 1 of say_hi
Hi!
Call 2 of say_hi
Hi!


#### Class-based decorator explained:

What happens when you use `@CountCalls`

```
@CountCalls
def say_hi():
    print("Hi!")
```

This is equivalent to:

```
def say_hi():
    print("Hi!")

say_hi = CountCalls(say_hi)
```
This creates an instance of CountCalls and calls `__init__`:

```
    def __init__(self, func):
        self.func = func    # Store the original say_hi function
        self.count = 0      # init counter = 0
```

Now say_hi is a CountCalls object that looks like:

```
say_hi = CountCalls instance {
    func: <original say_hi function>,
    count: 0
}
```

First call to say_hi(), it looks for the `__call__` method:

```
    def __call__(self, *args, **kwargs):
        self.count += 1     # self.count goes from 0 to 1
        print(f"Call {self.count} of {self.func.__name__}")
        return self.func(*args, **kwargs)
```

Now the object that looks like:

```
say_hi = CountCalls instance {
    func: <original say_hi function>,
    count: 1
}
```

Second call to say_hi(), again `__call__` is invoked on the same object, count goes from 1 to 2

the object now looks like:
```
say_hi = CountCalls instance {
    func: <original say_hi function>,
    count: 2
}
```

Same thing with the following example:

In [ ]:
# Regular class example
class BankAccount:
    def __init__(self, balance):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

account = BankAccount(100)
account.deposit(50)  # balance is now 150
account.deposit(30)  # balance is now 180
# The balance persists between calls!

Also the magic of `__call__`

It's a special method that makes an object callable (able to be called like a function):

In [81]:
class AddIt:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return x + self.n

add_5 = AddIt(5)
add_5(10)


15

If no `__call__`, then you have to:

In [80]:
class AddIt:
    def __init__(self, n):
        self.n = n
    
    def add(self, x):
        return x + self.n

add_5 = AddIt(5)
add_5.add(10)

15

### Stacking decorators
You can apply multiple decorators to one function:

In [ ]:
@decorator1
@decorator2
def my_function():
    pass

# Equivalent to: my_function = decorator1(decorator2(my_function))